# Image processing for GTVp reconstruction

## Requirements

In [ ]:
import numpy as np
import skimage
from skimage.morphology import binary_opening, binary_closing, disk, ball
from matplotlib.widgets import Slider
import matplotlib.pyplot as plt
import os
import nibabel as nib
import gzip
from skimage.measure import label
import ipywidgets as widgets
from IPython.display import display
from matplotlib.lines import Line2D

## Read in GTVp structures

In [ ]:
def read_nifti_data(file_path):
    """
    Read a NIfTI file and return the image data and voxel size.
    This function handles both plain NIfTI files and gzipped NIfTI files.
    
    Parameters
    ----------
    file_path : str
        Path to the NIfTI file (can be gzipped, e.g. ending with '.gz').
        
    Returns
    -------
    tuple
        (image_data, voxel_size) where image_data is a numpy.ndarray and voxel_size 
        is a tuple of voxel dimensions.
    """
    
    def load_nifti_file(path):
        """Load a NIfTI file using nibabel."""
        nifti_img = nib.load(path)
        img_data = nifti_img.get_fdata()
        voxel_size = nifti_img.header.get_zooms()
        return img_data, voxel_size

    # If the file is gzipped, extract and read it.
    if file_path.endswith('.gz'):
        try:
            with gzip.open(file_path, 'rb') as f_in:
                file_content = f_in.read()
        except Exception as e:
            print(f"Error opening gzip file '{file_path}': {e}")
            return None, None
        
        # Write the content to a temporary file.
        temp_filename = 'temp_nifti.nii'
        with open(temp_filename, 'wb') as temp_file:
            temp_file.write(file_content)
        
        # Debug: check file size.
        temp_size = os.path.getsize(temp_filename)
        if temp_size == 0:
            print(f"Error: Temporary file '{temp_filename}' is empty!")
            return None, None
        else:
            print(f"Temporary file '{temp_filename}' written successfully ({temp_size} bytes).")
        
        # Load the NIfTI data from the temporary file.
        image_data, voxel_size = load_nifti_file(temp_filename)
        os.remove(temp_filename)  # Clean up the temporary file.
        return image_data, voxel_size
    else:
        # If not gzipped, load directly.
        return load_nifti_file(file_path)

## Image processing

In [ ]:

def binary_union(arr1, arr2):
    """
    Compute the union of two binary 3D arrays.
    
    Parameters
    ----------
    arr1 : numpy.ndarray
        First binary 3D array.
    arr2 : numpy.ndarray
        Second binary 3D array.
        
    Returns
    -------
    numpy.ndarray
        A binary 3D array where each voxel is True if it is True in either arr1 or arr2.
    """
    if arr1.shape != arr2.shape:
        raise ValueError("The input arrays must have the same shape.")
    # Use logical OR and convert back to binary (0,1) if needed.
    union = np.logical_or(arr1, arr2)
    return union.astype(arr1.dtype)

def binary_subtract(arr1, arr2):
    """
    Subtract one binary 3D array from another.
    
    Parameters
    ----------
    arr1 : numpy.ndarray
        The binary 3D array from which to subtract.
    arr2 : numpy.ndarray
        The binary 3D array to subtract.
        
    Returns
    -------
    numpy.ndarray
        A binary 3D array where a voxel is True only if it is True in arr1 and False in arr2.
    """
    if arr1.shape != arr2.shape:
        raise ValueError("The input arrays must have the same shape.")
    # A minus B: True only when A is True and B is False.
    subtracted = np.logical_and(arr1, np.logical_not(arr2))
    return subtracted.astype(arr1.dtype)

def overlay_label_and_image(original_image, labeled_array, title="Overlay of Labeled Components"):
    """
    Display an interactive widget that overlays a labeled array on top of an original image.
    
    The original image is displayed in grayscale and the labeled array is overlaid using a colormap 
    with transparency. Hovering over the image shows pixel coordinates and the label value.
    
    Parameters
    ----------
    original_image : numpy.ndarray
        A 3D image array (e.g. grayscale) with shape (height, width, slices).
    labeled_array : numpy.ndarray
        A 3D labeled array with the same shape as original_image, where each connected component 
        has a unique integer label.
    title : str, optional
        Title for the displayed plot.
    """
    if original_image.shape != labeled_array.shape:
        raise ValueError("Original image and labeled array must have the same shape.")
    
    num_slices = original_image.shape[2]
    
    def view_slice(slice_index):
        fig, ax = plt.subplots(figsize=(8,8))
        # Plot original image in grayscale
        ax.imshow(original_image[:, :, slice_index], cmap='gray', interpolation='none')
        # Overlay the labeled array with transparency and a colormap
        im = ax.imshow(labeled_array[:, :, slice_index], cmap='nipy_spectral', alpha=0.5, interpolation='none')
        # Add colorbar for the labeled overlay
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label("Label")
        
        # Custom coordinate formatter to display label values on hover
        def format_coord(x, y):
            col = int(round(x))
            row = int(round(y))
            if 0 <= col < labeled_array.shape[1] and 0 <= row < labeled_array.shape[0]:
                label_val = labeled_array[row, col, slice_index]
                return f"x={col}, y={row}, slice={slice_index}, label={label_val}"
            else:
                return f"x={col}, y={row}, slice={slice_index}"
        ax.format_coord = format_coord
        
        ax.set_title(f"{title} - Slice {slice_index}")
        ax.axis('off')
        plt.show()
    
    slider = widgets.IntSlider(min=0, max=num_slices-1, step=1, value=num_slices//2, description='Slice')
    display(widgets.interact(view_slice, slice_index=slider))

def image_widget(image, contour1, contour2, title="Image"):
    """
    Display a scrollable slider for a 3D image where one can hover over the pixels and see the values.

    Parameters:
    image (numpy.ndarray): The 3D image data.
    """
    num_slices = image.shape[2]

    def view_slice(slice_index):

        plt.figure(figsize=(10, 10))
        plt.imshow(image[:, :, slice_index], cmap='gray')
        plt.contour(contour1[:, :, slice_index], colors='yellow')
        if contour2 is not None:
            plt.contour(contour2[:, :, slice_index], colors='blue')
        custom_lines = [
            Line2D([0], [0], color='yellow', lw=2, label='Contour 1'),
            Line2D([0], [0], color='blue', lw=2, label='Contour 2')]
        plt.legend(handles=custom_lines, loc='upper right')
        plt.title(f"{title} - Slice {slice_index}")
        plt.axis('off')
        plt.show()

    slice_slider = widgets.IntSlider(min=num_slices - 150, max=num_slices - 1, step=1, description='Slice')
    widgets.interact(view_slice, slice_index=slice_slider)


def image_processing(orig_folderpath, filename_1, filename_2=None, output_path=None,
                     substract=False, union=False, labeling=False, 
                     gtvp_label=None, filename='GTVp.nii.gz'):
    """
    Process image data from a folder. Depending on the parameters, the function performs
    binary union, subtraction, or connected-component labeling on the input images.
    
    Parameters
    ----------
    orig_folderpath : str
        Folder path containing the image files.
    filename_1 : str
        Filename for the first image.
    filename_2 : str, optional
        Filename for the second image (if performing union or subtraction).
    output_path : str, optional
        Directory where the processed image will be saved.
    substract : bool, optional
        If True, subtracts image2 from image1.
    union : bool, optional
        If True, computes the union of image1 and image2.
    labeling : bool, optional
        If True, performs connected-component labeling on image1.
    gtvp_label : list, optional
        List of integer labels to extract from the labeled image (if labeling is True).
    filename : str, optional
        Filename to save the processed image (default 'GTVp.nii.gz').
    
    Returns
    -------
    numpy.ndarray
        The processed image array.
    """
    # Create full output path and ensure directory exists.
    filepath = os.path.join(output_path, filename)
    os.makedirs(output_path, exist_ok=True)
    
    # Load the 'image.nii.gz' (for display purposes)
    image, voxel_site_image = read_nifti_data(os.path.join(orig_folderpath, 'image.nii.gz'))
    image = np.transpose(image, (1, 0, 2))
    
    # Load the first image.
    image_data_1, voxel_size_1 = read_nifti_data(os.path.join(orig_folderpath, filename_1))
    image_data_1 = np.transpose(image_data_1, (1, 0, 2))
    image_widget(image, image_data_1, None, title="Image")
    
    # Optionally load a second image.
    if filename_2 is not None:
        image_data_2, voxel_size_2 = read_nifti_data(os.path.join(orig_folderpath, filename_2))
        image_data_2 = np.transpose(image_data_2, (1, 0, 2))
        image_widget(image, image_data_1, image_data_2, title="Image")
    
    # Perform union if requested.
    if union:
        image_processed = binary_union(image_data_1, image_data_2)
        # Save and display the result.
        save_nifti_gz(np.transpose(image_processed, (1, 0, 2)), filepath, voxel_size=voxel_size_1)
        image_widget(image, image_processed, None, title="Union Image")
        return image_processed
    
    # Perform subtraction if requested.
    if substract:
        image_processed = binary_subtract(image_data_1, image_data_2)
        save_nifti_gz(np.transpose(image_processed, (1, 0, 2)), filepath, voxel_size=voxel_size_1)
        image_widget(image, image_processed, None, title="Subtraction Image")
        return image_processed
    
    # Perform labeling if requested.
    if labeling:
        labeled_image = skimage.measure.label(image_data_1, connectivity=1)
        # Display overlay: original image with labeled components on top.
        overlay_label_and_image(image, labeled_image, title="Overlay of Labeled Components")
        if gtvp_label is not None:
            mask_gtvp = np.zeros_like(labeled_image)
            for lab in gtvp_label:
                # Compare with the current label (lab) from the list.
                mask = (labeled_image == lab)
                masked_image = image_data_1 * mask
                mask_gtvp = binary_union(mask_gtvp, masked_image)
            image_processed = image_data_1 * mask_gtvp
            image_widget(image, image_processed, None, title="Labeled & Masked Image")
            save_nifti_gz(np.transpose(image_processed, (1, 0, 2)), filepath, voxel_size=voxel_size_1)
            return image_processed
        else:
            # If no specific gtvp_label list is provided, just return the labeled image.
            return labeled_image


%matplotlib widget
#%matplotlib inline

# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/10683066/renamed1"
# filename1 = r"mask_GTV1_PT_70Gy_1a.nii.gz"
# filename2 = r"mask_GTV2_PT_70Gy_1a.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/10683066"
# image_processing(orig_path, filename1, filename2, output_path = output_path, union = True, substract=False, filename = 'GTVp.nii.gz')

# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/10779681/renamed1"
# filename1 = r"mask_GTVges.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/10779681"
# image_processing(orig_path, filename1, output_path = output_path, labeling = True, gtvp_label= [2], filename = 'GTVp.nii.gz')


# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/01086588/renamed1"
# filename1 = r"mask_GTV_70Gy_V1_1a.nii.gz"
# filename2 = r"mask_GTV_70Gy_V2_1a.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/01086588"
# image_processing(orig_path, filename1, filename2, output_path = output_path, union = True, substract=False, filename = 'GTVp.nii.gz')

# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/03339327/renamed1"
# filename1 = r"mask_GTVges.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/03339327"
# image_processing(orig_path, filename1, output_path = output_path, labeling = True, gtvp_label= [1,3], filename = 'GTVp.nii.gz')


# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/10701864/renamed1"
# filename1 = r"mask_GTV-ges.nii.gz"
# filename2 = r"mask_GTV LN_70Gy_1a.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/10701864"
# image_processing(orig_path, filename1, filename2, output_path = output_path, union = False, substract=True, filename = 'GTVp.nii.gz')

# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/01982869/renamed1"
# filename1 = r"mask_GTV_CT.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/01982869"
# image_processing(orig_path, filename1, output_path = output_path, labeling = True, gtvp_label= [1], filename = 'GTVp.nii.gz')



# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/00873438/renamed1"
# filename1 = r"mask_GTV_70Gy_1a.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/00873438"
# image_processing(orig_path, filename1, output_path = output_path, labeling = True, gtvp_label= [1], filename = 'GTVp.nii.gz')

# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/07370784/renamed1"
# filename1 = r"mask_GTV1_V1_1a.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/07370784"
# image_processing(orig_path, filename1, output_path = output_path, labeling = True, gtvp_label= [1], filename = 'GTVp.nii.gz')



# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/10859618/renamed2"
# filename1 = r"mask_GTV1_V1_1a.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/10859618"
# image_processing(orig_path, filename1, output_path = output_path, labeling = True, gtvp_label= [1], filename = 'GTVp.nii.gz')


# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/10754577/renamed1"
# filename1 = r"mask_gtvp.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/10754577"
# image_processing(orig_path, filename1, output_path = output_path, labeling = True, gtvp_label= [1], filename = 'GTVp.nii.gz')

# orig_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/06_midline_extraction/10852680/renamed2"
# filename1 = r"mask_GTV.nii.gz"
# output_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Image_processed_patients_gtvp/10852680"
# image_processing(orig_path, filename1, output_path = output_path, labeling = True, gtvp_label= [1], filename = 'GTVp.nii.gz')



## Save np.array as nifti (nii.gz)

In [ ]:

def save_nifti_gz(image_data, file_path, affine=None, voxel_size=None):
    """
    Save a 3D image (numpy.ndarray) as a gzipped NIfTI file (.nii.gz).

    Parameters
    ----------
    image_data : numpy.ndarray
        The image data to be saved.
    file_path : str
        The output file path. It should end with '.nii.gz'.
    affine : numpy.ndarray, optional
        The affine transformation matrix (4x4). If None, an identity matrix is used.
    voxel_size : tuple, optional
        A tuple of voxel dimensions (e.g., (s_x, s_y, s_z)). If provided, it is used to set 
        the zooms in the header.
        
    Returns
    -------
    None
    """
    if affine is None:
        affine = np.eye(4)
    
    # Create a NIfTI image. This will automatically create a header based on image_data.
    nifti_img = nib.Nifti1Image(image_data, affine)
    
    # If voxel_size is provided, update the header's zooms.
    if voxel_size is not None:
        nifti_img.header.set_zooms(voxel_size)
    
    # Save the image; the '.nii.gz' extension tells nibabel to gzip it.
    nib.save(nifti_img, file_path)
    print(f"Image saved as {file_path}")

# Example usage:
# Assuming image_data is a numpy array and voxel_size is a tuple like (1.0, 1.0, 1.5)
# save_nifti_gz(image_data, 'output_image.nii.gz', voxel_size=(1.0, 1.0, 1.5))
